##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 简单的音频识别：识别关键词

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/audio/simple_audio"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/audio/simple_audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/audio/simple_audio.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/audio/simple_audio.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>
</td>
</table>

本教程演示了如何预处理 WAV 格式的音频文件，并构建和训练一个基本的<a href="https://en.wikipedia.org/wiki/Speech_recognition" class="external">自动语音识别</a> (ASR) 模型来识别十个不同的单词。您将使用 [Speech Commands 数据集](https://www.tensorflow.org/datasets/catalog/speech_commands)（<a href="https://arxiv.org/abs/1804.03209" class="external">Warden，2018 年</a>）的一部分，其中包含命令的短（一秒或更短）音频片段，例如“down”、“go”、“left”、“no”、“right”、“stop”、“up”和“yes”。

现实世界的语音和音频识别<a href="https://ai.googleblog.com/search/label/Speech%20Recognition" class="external">系统</a>很复杂。但是，就像[使用 MNIST 数据集进行图像分类](../quickstart/beginner.ipynb)一样，本教程应该能够使您对所涉及的技术有一个基本的了解。

## 设置

导入必要的模块和依赖项。请注意，您将在本教程中使用 <a href="https://seaborn.pydata.org/" class="external">seaborn</a> 进行可视化。

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## 导入迷你 Speech Commands 数据集

为了节省数据加载时间，您将使用较小版本的 Speech Commands 数据集。[原始数据集](https://www.tensorflow.org/datasets/catalog/speech_commands)包含超过 105,000 个音频文件，采用 <a href="https://www.aelius.com/njh/wavemetatools/doc/riffmci.pdf" class="external">WAV（波形）音频文件格式</a>，内容是不同的人们说出 35 个不同的单词。此数据由 Google 收集并根据 CC BY 许可发布。

使用 `tf.keras.utils.get_file` 下载并提取包含较 Speech Commands 数据集的 `mini_speech_commands.zip`：

In [ ]:
DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

数据集的音频片段存储在与每个语音命令对应的八个文件夹中：`no`、`yes`、`down`、`go`、`left`、`up`、`right` 和 `stop`：

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[commands != 'README.md']
print('Commands:', commands)

将音频片段提取到名为 `filenames` 的列表中，然后对其进行乱序：

In [ ]:
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
filenames = tf.random.shuffle(filenames)
num_samples = len(filenames)
print('Number of total examples:', num_samples)
print('Number of examples per label:',
      len(tf.io.gfile.listdir(str(data_dir/commands[0]))))
print('Example file tensor:', filenames[0])

分别使用 80:10:10 的比例将 `filenames` 拆分为训练集、验证集和测试集：

In [ ]:
train_files = filenames[:6400]
val_files = filenames[6400: 6400 + 800]
test_files = filenames[-800:]

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

## 读取音频文件及其标签

在本部分，您将预处理数据集，为波形和相应的标签创建解码张量。注意：

- 每个 WAV 文件都包含时间序列数据，每秒具有一定数量的样本。
- 每个样本代表该特定时间的音频信号的<a href="https://en.wikipedia.org/wiki/Amplitude" class="external">幅度</a>。
- 在 <a href="https://en.wikipedia.org/wiki/Audio_bit_depth" class="external">16 位</a>系统（如 mini Speech Commands 数据集中的 WAV 文件）中，幅度值范围为 -32,768 到 32,767。
- 该数据集的<a href="https://en.wikipedia.org/wiki/Sampling_(signal_processing)#Audio_sampling" class="external">采样率为 </a>16kHz。

`tf.audio.decode_wav` 返回的张量的形状是 `[samples, channels]`，其中 `channels` 为 `1` 表示单声道，`2` 表示立体声。迷你 Speech Commands 数据集仅包含单声道录音。 

In [ ]:
test_file = tf.io.read_file(DATASET_PATH+'/down/0a9f9af7_nohash_0.wav')
test_audio, _ = tf.audio.decode_wav(contents=test_file)
test_audio.shape

接下来，我们来定义一个函数，将数据集的原始 WAV 音频文件预处理为音频张量：

In [ ]:
def decode_audio(audio_binary):
  # Decode WAV-encoded audio files to `float32` tensors, normalized
  # to the [-1.0, 1.0] range. Return `float32` audio and a sample rate.
  audio, _ = tf.audio.decode_wav(contents=audio_binary)
  # Since all the data is single channel (mono), drop the `channels`
  # axis from the array.
  return tf.squeeze(audio, axis=-1)

定义一个使用每个文件的父目录创建标签的函数：

- 将文件路径拆分为 `tf.RaggedTensor`（具有不规则维度的张量 - 带有可能具有不同长度的切片）。

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(
      input=file_path,
      sep=os.path.sep)
  # Note: You'll use indexing here instead of tuple unpacking to enable this
  # to work in a TensorFlow graph.
  return parts[-2]

定义另一个辅助函数 `get_waveform_and_label`，将它们整合在一起：

- 输入是 WAV 音频文件名。
- 输出是一个元组，其中包含准备好进行监督学习的音频和标签张量。

In [ ]:
def get_waveform_and_label(file_path):
  label = get_label(file_path)
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform, label

构建训练集以提取音频标签对：

- 使用前面定义的 `get_waveform_and_label`，创建具有 `Dataset.from_tensor_slices` 和 `Dataset.map` 的 `tf.data.Dataset`。

稍后您将使用类似的过程构建验证集和测试集。

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

files_ds = tf.data.Dataset.from_tensor_slices(train_files)

waveform_ds = files_ds.map(
    map_func=get_waveform_and_label,
    num_parallel_calls=AUTOTUNE)

我们来绘制一些音频波形：

In [ ]:
rows = 3
cols = 3
n = rows * cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 12))

for i, (audio, label) in enumerate(waveform_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  ax.plot(audio.numpy())
  ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
  label = label.numpy().decode('utf-8')
  ax.set_title(label)

plt.show()

## 将波形转换为频谱图

数据集中的波形在时域中表示。接下来，您将通过计算<a href="https://en.wikipedia.org/wiki/Short-time_Fourier_transform" class="external">短时傅里叶变换 (STFT)</a> 将波形从时域信号转换为时频域信号，以将波形转换为<a href="https://en.wikipedia.org/wiki/Spectrogram" clas="external">频谱图</a>，显示频率随时间的变化，并且可以表示为二维图像。您将把频谱图图像输入您的神经网络以训练模型。

傅里叶变换 (`tf.signal.fft`) 会将信号转换为其分量频率，但会丢失所有时间信息。相比之下，STFT (`tf.signal.stft`) 会将信号拆分为时间窗口，并在每个窗口上运行傅里叶变换，保留一些时间信息，并返回可以运行标准卷积的二维张量。

创建用于将波形转换为频谱图的效用函数：

- 这些波形需要具有相同的长度，以便将它们转换为频谱图时，结果具有相似的维度。这可以通过简单地对短于一秒的音频片段进行零填充（使用 `tf.zeros`）来完成。
- 调用 `tf.signal.stft` 时，请选择 `frame_length` 和 `frame_step` 参数，使生成的频谱图“图像”几乎为方形。有关 STFT 参数选择的更多信息，请参阅有关音频信号处理和 STFT 的 <a href="https://www.coursera.org/lecture/audio-signal-processing/stft-2-tjEQe" class="external">Coursera 视频</a>。
- STFT 会产生表示幅度和相位的复数数组。但是，在本教程中，您将只使用幅度，您可以通过在 tf.signal.stft 的输出上应用 `tf.abs` 来获得该 `tf.signal.stft`。

In [ ]:
def get_spectrogram(waveform):
  # Zero-padding for an audio waveform with less than 16,000 samples.
  input_len = 16000
  waveform = waveform[:input_len]
  zero_padding = tf.zeros(
      [16000] - tf.shape(waveform),
      dtype=tf.float32)
  # Cast the waveform tensors' dtype to float32.
  waveform = tf.cast(waveform, dtype=tf.float32)
  # Concatenate the waveform with `zero_padding`, which ensures all audio
  # clips are of the same length.
  equal_length = tf.concat([waveform, zero_padding], 0)
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

接下来，开始探索数据。打印一个样本的张量波形形状和相应的频谱图，并播放原始音频：

In [ ]:
for waveform, label in waveform_ds.take(1):
  label = label.numpy().decode('utf-8')
  spectrogram = get_spectrogram(waveform)

print('Label:', label)
print('Waveform shape:', waveform.shape)
print('Spectrogram shape:', spectrogram.shape)
print('Audio playback')
display.display(display.Audio(waveform, rate=16000))

现在，定义一个显示频谱图的函数：

In [ ]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

绘制样本随时间变化的波形和相应的频谱图（随时间变化的频率）：

In [ ]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])

plot_spectrogram(spectrogram.numpy(), axes[1])
axes[1].set_title('Spectrogram')
plt.show()

接下来，定义一个函数，将波形数据集转换为频谱图，并将其对应的标签作为整数 ID：

In [ ]:
def get_spectrogram_and_label_id(audio, label):
  spectrogram = get_spectrogram(audio)
  label_id = tf.math.argmax(label == commands)
  return spectrogram, label_id

使用 `Dataset.map` 在数据集的元素之间映射 `get_spectrogram_and_label_id`：

In [ ]:
spectrogram_ds = waveform_ds.map(
  map_func=get_spectrogram_and_label_id,
  num_parallel_calls=AUTOTUNE)

检查数据集不同样本的频谱图：

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

for i, (spectrogram, label_id) in enumerate(spectrogram_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  plot_spectrogram(spectrogram.numpy(), ax)
  ax.set_title(commands[label_id.numpy()])
  ax.axis('off')

plt.show()

## 构建并训练模型

对验证集和测试集重复训练集预处理：

In [ ]:
def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)
  output_ds = files_ds.map(
      map_func=get_waveform_and_label,
      num_parallel_calls=AUTOTUNE)
  output_ds = output_ds.map(
      map_func=get_spectrogram_and_label_id,
      num_parallel_calls=AUTOTUNE)
  return output_ds

In [ ]:
train_ds = spectrogram_ds
val_ds = preprocess_dataset(val_files)
test_ds = preprocess_dataset(test_files)

批处理用于模型训练的训练集和验证集：

In [ ]:
batch_size = 64
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

添加 `Dataset.cache` 和 `Dataset.prefetch` 运算以减少训练模型时的读取延迟：

In [ ]:
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)

对于模型，您将使用简单的卷积神经网络 (CNN)，因为您已将音频文件转换为频谱图图像。

您的 `tf.keras.Sequential` 模型将使用以下 Keras 预处理层：

- `tf.keras.layers.Resizing`：对输入进行下采样以使模型训练得更快。
- `tf.keras.layers.Normalization`：根据图像的均值和标准差对图像中的每个像素进行归一化。

对于 `Normalization` 层，首先需要在训练数据上调用其 `adapt` 方法，以计算聚合统计数据（即均值和标准差）。

In [ ]:
for spectrogram, _ in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)
num_labels = len(commands)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
norm_layer.adapt(data=spectrogram_ds.map(map_func=lambda spec, label: spec))

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

使用 Adam 优化器和交叉熵损失配置 Keras 模型：

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

出于演示目的，将模型训练超过 10 个周期：

In [ ]:
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

我们来绘制训练和验证损失曲线，以检查您的模型在训练期间的改进情况：

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

## 评估模型性能

在测试集上运行模型并检查模型的性能：

In [ ]:
test_audio = []
test_labels = []

for audio, label in test_ds:
  test_audio.append(audio.numpy())
  test_labels.append(label.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

In [ ]:
y_pred = np.argmax(model.predict(test_audio), axis=1)
y_true = test_labels

test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy: {test_acc:.0%}')

### 显示混淆矩阵

使用<a href="https://developers.google.com/machine-learning/glossary#confusion-matrix" class="external">混淆矩阵</a>检查模型对测试集中每个命令的分类效果：


In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=commands,
            yticklabels=commands,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

## 对音频文件运行推断

最后，使用某人说“no”的输入音频文件验证模型的预测输出。您的模型表现如何？

In [ ]:
sample_file = data_dir/'no/01bb6a2a_nohash_0.wav'

sample_ds = preprocess_dataset([str(sample_file)])

for spectrogram, label in sample_ds.batch(1):
  prediction = model(spectrogram)
  plt.bar(commands, tf.nn.softmax(prediction[0]))
  plt.title(f'Predictions for "{commands[label[0]]}"')
  plt.show()

如输出所示，您的模型应该已将音频命令识别为“no”。

## 后续步骤

本教程演示了如何使用带有 TensorFlow 和 Python 的卷积神经网络执行简单的音频分类/自动语音识别。要了解更多信息，请考虑以下资源：

- [使用 YAMNet 进行声音分类](https://www.tensorflow.org/hub/tutorials/yamnet)教程展示了如何使用迁移学习进行音频分类。
- 来自 <a href="https://www.kaggle.com/c/tensorflow-speech-recognition-challenge/overview" class="external">Kaggle 的 TensorFlow 语音识别挑战</a>的笔记本。
- <a href="https://codelabs.developers.google.com/codelabs/tensorflowjs-audio-codelab/index.html#0" class="external">TensorFlow.js - 使用迁移学习的音频识别代码实验室</a>教授如何构建您自己的交互式 Web 应用以进行音频分类。
- arXiv 上的<a href="https://arxiv.org/abs/1709.04396" class="external">音乐信息检索深度学习教程</a>（Choi 等人，2017 年）。
- TensorFlow 还为[音频数据准备和增强](https://www.tensorflow.org/io/tutorials/audio)提供额外支持，以帮助您完成基于音频的项目。
- 请考虑使用 <a href="https://librosa.org/" class="external">librosa</a> 库，这是一个用于音乐和音频分析的 Python 包。